Script Objective: To delineate watersheds from a Digital Elevation Model (DEM) and create a shapefile containing the watershed polygons.

Work plan:
1. Data acquisition
    - Obtaining DEM: Acquire a suitable Digital Elevation Model (DEM) for tests.
3. Environment setup
    - Installing required libraries and Jupyter Notebook
4. Script writing
    - Load the DEM
    - Identify stream network
    - Identify peaks and ridges in the DEM
    - Delineate watersheds
    - Convert to shapefile

In [141]:
#Importing necessary modules
import geopandas as gpd
import rasterio as rio
import skimage as ski
import scipy as sp
import numpy as np

In [142]:
#Opening the DEM data
dem = rio.open(r'C:\Users\rrztscno\Downloads\7950_3050.tif')

#Reading the first band (elevation data)
dem_array = dem.read(1)

#Getting the transformation and coordinate reference system
transform = dem.transform
crs = dem.crs

In [143]:
#Creating a binary mask (0 for nodata, 1 for valid data)
binary_mask = dem_array > 0

#Removing small holes in the mask
filled_mask = ski.morphology.remove_small_holes(binary_mask)

#Obtaining a dem excluding nodata values
filled_dem = filled_mask * dem_array

In [144]:
#Calculating the 5th percentile threshold for stream delineation (lower values are the streams)
percentile = 5
threshold = np.percentile(filled_dem, percentile)

#Creating a stream network based on the threshold
stream_mask = dem_array < threshold

In [145]:
#Generate markers for watershed segmentation
markers = sp.ndimage.label(stream_mask)[1]

#Performing watershed segmentation using the negative filled DEM (streams become ridges and ridges become streams)
labels = ski.segmentation.watershed(filled_dem, markers)

In [146]:
#Converting watershed labels to GeoDataFrame features
from rasterio import features
ws_geoms = features.shapes(labels, transform=transform)
ws_features = []
for geom, attr in ws_geoms:
    ws_features.append({'geometry': geom,'properties': None})
watersheds = gpd.GeoDataFrame.from_features(ws_features, crs=crs)

In [147]:
#Saving the watersheds as a shapefile
watersheds.to_file(r'C:\Users\rrztscno\Downloads\watersheds.shp')

Future work plan: 
1. Quality control with different DEMs and areas to evaluate effectiveness
2. Possibility of adding stream network shapefile in the output
3. Compile all code into a python function with the DEM as argument